In [32]:
import pandas as pd
import random
import itertools
import numpy as np
import time 
import pickle
from scipy.stats import ttest_ind, chi2_contingency, fisher_exact, ranksums
from scipy.stats import pearsonr, spearmanr

# Context and Background

**this analysis will only be done on tumor cells**

1. For each of the 22 genes on our short list, we are going to generate a list of the top 1000 other genes that covary with it the most. So for example, for GABRP, we will want to know which 1000 genes out of the entire transcriptome (so the whole matrix) have the highest correlation to it. Then I will run biological analyses on this list of 1000 genes that you give me.

2. I am going to discuss in terms of GABRP because I think it will be easier just to use a specific gene as a case example. First, we will subset only to the cells that contain >0 expression of GABRP. This will hopefully make the computational analysis more streamlined by reducing the size of the dataset to work with.

3. Then, for each two gene correlation assessment (e.g. GABRP and gene X), we will first evaluate if there are at least 100 cells in which there is non-zero expression for both GABRP and gene X. This is to isolate our analysis specifically to cells in which there are continuous values present, so that we don't have zero values tampering with our data. Do this for each combination of GABRP and every other gene remaining in the matrix - hopefully this will make the dataset smaller again.

4. Now, in our matrix, we should only have GABRP and the genes for which there are at least 100 cells containing continuous expression values. Now for each pair of GABRP and gene X, let's generate a Pearson coefficient and Spearman's coefficient (both the R/p and also the p value), only on the cells that have continuous variables.


In [2]:
column_names = pd.read_csv('data/columns for X.csv',header = None)
column_names = list(column_names[0])

rows = pd.read_csv('data/rows for X.csv',header = None)
cell_type = list(rows[0])
cell_type = [val.lower() for val in cell_type]

gene_list_22 = pd.read_csv('data/gene_list_22.csv', header = None)
gene_list_22 = list(gene_list_22[0])

In [ ]:
df_22 = pd.read_csv('data/X.csv',names = column_names, usecols = gene_list_22)
df_22 = df_22[gene_list_22]
df_22['cell_type'] = cell_type
df_22_tumor = df_22[df_22['cell_type'] == 'tumor']
df_22_tumor.shape
df_22_tumor.replace(0,np.nan, inplace = True)
df_22_tumor_dropna = df_22_tumor.dropna(how = 'all', subset = gene_list_22)
df_22_tumor_dropna.shape

In [33]:
index_dict = {}
for gene in gene_list_22:
    index_dict[gene] = list(df_22_tumor[~df_22_tumor[gene].isna()].index)

In [36]:
pickle.dump(index_dict, open("data/gene_22_index_dict.pkl", "wb"))  # save it into a file named save.p

In [3]:
with open("data/gene_22_index_dict.pkl", "rb") as f:
    index_dict = pickle.load(f)

In [37]:

gene_col_list = []
r_pearson_list = []
r_spearman_list = []
p_pearson_list = []
p_spearman_list = []
n_rows_list = []

for col in df_g.columns:
    filename = '{}_correlation.csv'.format(gene)
    if col not in gene:
        df_subset = df_g[[gene,col]]
        df_subset_filtered = df_subset[~df_subset[col].isna()]
        r_pearson, p_pearson = pearsonr(df_subset_filtered[gene], df_subset_filtered[col])
        r_spearman, p_spearman = spearmanr(df_subset_filtered[gene], df_subset_filtered[col])
        
        n_rows = df_subset_filtered.shape[0]
        gene_col_list.append(col)
        r_pearson_list.append(r_pearson)
        p_pearson_list.append(p_pearson)
        
        r_spearman_list.append(r_spearman)
        p_spearman_list.append(p_spearman)        
        
        n_rows_list.append(n_rows)


In [39]:
df_gene_corr = pd.DataFrame({'gene':gene_col_list,
                             'r_pearson':np.round(r_pearson_list,5),
                             'p_pearson':p_pearson_list,
                             'r_spearman':np.round(r_spearman_list,5),
                             'p_spearman':p_spearman_list,
                             'n_rows_nonzero' : n_rows_list
                            })
df_gene_corr.sort_values(by= 'r_pearson', ascending = False, inplace = True)
df_gene_corr.to_csv('data/single_cell_correlation/{}'.format(filename),index = False)

In [ ]:
for gene in gene_list_22:
    
    skiprows = [i for i in np.arange(88031) if i not in index_dict[gene]]
    df_g = pd.read_csv('data/X.csv',names = column_names, skiprows = skiprows)
    df_g.replace(0,np.nan, inplace = True)
    df_g.dropna(axis = 1, thresh = 100, inplace = True)
    
    gene_col_list = []
    r_pearson_list = []
    r_spearman_list = []
    p_pearson_list = []
    p_spearman_list = []
    n_rows_list = []

    for col in df_g.columns:
        filename = '{}_correlation.csv'.format(gene)
        if col not in gene:
            df_subset = df_g[[gene,col]]
            df_subset_filtered = df_subset[~df_subset[col].isna()]
            r_pearson, p_pearson = pearsonr(df_subset_filtered[gene], df_subset_filtered[col])
            r_spearman, p_spearman = spearmanr(df_subset_filtered[gene], df_subset_filtered[col])

            n_rows = df_subset_filtered.shape[0]
            gene_col_list.append(col)
            r_pearson_list.append(r_pearson)
            p_pearson_list.append(p_pearson)

            r_spearman_list.append(r_spearman)
            p_spearman_list.append(p_spearman)        

            n_rows_list.append(n_rows)
            
    df_gene_corr = pd.DataFrame({'gene':gene_col_list,
                                 'r_pearson':np.round(r_pearson_list,5),
                                 'p_pearson':p_pearson_list,
                                 'r_spearman':np.round(r_spearman_list,5),
                                 'p_spearman':p_spearman_list,
                                 'n_rows_nonzero' : n_rows_list
                                })
    df_gene_corr.sort_values(by= 'r_pearson', ascending = False, inplace = True)
    df_gene_corr.to_csv('data/single_cell_correlation/{}'.format(filename),index = False)
    print(gene)